In [ ]:
import numpy as np
import scipy
import pandas as pd
import time
import random
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
#! pip install sentence-transformers
'''! pip install datasets
! pip install accelerate -U
! pip install transformers[torch]
! pip install --upgrade pip
! pip install transformers[torch] accelerate -U
! python -c "import transformers; print(transformers.__version__)"
! python -c "import accelerate; print(accelerate.__version__)"

SyntaxError: incomplete input (<ipython-input-2-d5273dfd0c32>, line 2)

In [ ]:
from google.colab import drive
import shutil
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/jawnson.csv'
destination_path = '/content/sample_data/'
shutil.copy(file_path, destination_path)

df = pd.read_csv('/content/sample_data/jawnson.csv')

# For copying a directory, use shutil.copytree
# shutil.copytree('/content/drive/My Drive/source_directory', '/content/destination_directory')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive.flush_and_unmount()
df.tail()

In [ ]:
#model = SentenceTransformer("all-distilroberta-v1")
# Load tokenizer and model
#tokenizer = DistilBertTokenizer.from_pretrained('distilroberta-base')
model = DistilBertForSequenceClassification.from_pretrained('distilroberta-base', num_labels=3)

You are using a model of type roberta to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.word_embeddings.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'transformer.layer.0.attention.k_lin.bias', 'transformer.layer.0.attention.k_lin.weight', 'transformer.layer.0.attention.out_lin.bias', 'transformer.layer.0.attention.out_lin.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.0.attention.q_lin.weight', 'transformer.layer.0.attention.v_lin.bias', 'transformer.layer.0.attention.v_lin.weight', 'transformer.layer.0.ffn.lin1.bias', 'transformer.layer.0.ffn.lin1.weight', 'transformer.layer.0.ffn.li

In [ ]:
#Test-train split
#change dataset to whatever the final cleaned data are called
'''dataset = Dataset.from_pandas(df)
train_testvalid = dataset.train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
train_dataset = train_testvalid['train']
eval_dataset = test_valid['train']'''

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
eval_dataset = dataset['test']

train_dataset.set_format(type='torch', columns=['bioname', 'nominate_dim1'])
eval_dataset.set_format(type='torch', columns=['bioname', 'nominate_dim1'])

train_dataset = train_dataset.rename_column("nominate_dim1", "labels")
eval_dataset = eval_dataset.rename_column("nominate_dim1", "labels")

train_dataset = train_dataset.rename_column("bioname", "input_ids")
eval_dataset = eval_dataset.rename_column("bioname", "input_ids")

In [ ]:
'''
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Apply tokenization to the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

ValueError: You have to specify either input_ids or inputs_embeds